### Retriever And chain with Langchain 


In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("attention.pdf")
docs = loader.load()
docs

Ignoring wrong pointing object 1098 0 (offset 0)
Ignoring wrong pointing object 1178 0 (offset 0)
Ignoring wrong pointing object 1239 0 (offset 0)
Ignoring wrong pointing object 1289 0 (offset 0)
Ignoring wrong pointing object 1303 0 (offset 0)
Ignoring wrong pointing object 1317 0 (offset 0)
Ignoring wrong pointing object 1330 0 (offset 0)


[Document(page_content='103C H A P T E R3Attention1.The Nature and Roles of Attention1.1.Failures of Selection1.1.1.Failures of Selection in Space1.1.2.Failures of Selection in Time1.1.3.Sources of LimitationDEBATE:Cars and Conversation1.1.4.Problems in Interpretation1.1.5.When the Brain Fails1.2.Successes of Selection1.2.1.Endogenous and ExogenousEffects in Space1.2.2.Components of Attention1.2.3.Cross-Modal Links1.2.4.Object-Based Attention2.Explaining Attention: Information ProcessingTheories2.1.Early versus Late Attentional Selection2.2.Spotlight Theory2.3.Feature Integration Theory and GuidedSearch3.Looking to the Brain3.1.Electrophysiology and Human Attention3.2.Functional Neuroimaging and TMS4.Competition: A Single Explanatory Frameworkfor Attention?A CLOSER LOOK:Competition and SelectionRevisit and ReflectLearning ObjectivesAt a very large and very noisy party, you’re looking for a friend you’ve lost in the crowd. Youlook for her green dress amid the sea of colors. You try to c

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1000,
    chunk_overlap = 20,)
text_splitter.split_documents(docs)[:5]

[Document(page_content='103C H A P T E R3Attention1.The Nature and Roles of Attention1.1.Failures of Selection1.1.1.Failures of Selection in Space1.1.2.Failures of Selection in Time1.1.3.Sources of LimitationDEBATE:Cars and Conversation1.1.4.Problems in Interpretation1.1.5.When the Brain Fails1.2.Successes of Selection1.2.1.Endogenous and ExogenousEffects in Space1.2.2.Components of Attention1.2.3.Cross-Modal Links1.2.4.Object-Based Attention2.Explaining Attention: Information ProcessingTheories2.1.Early versus Late Attentional Selection2.2.Spotlight Theory2.3.Feature Integration Theory and GuidedSearch3.Looking to the Brain3.1.Electrophysiology and Human Attention3.2.Functional Neuroimaging and TMS4.Competition: A Single Explanatory Frameworkfor Attention?A CLOSER LOOK:Competition and SelectionRevisit and ReflectLearning ObjectivesAt a very large and very noisy party, you’re looking for a friend you’ve lost in the crowd. Youlook for her green dress amid the sea of colors. You try to c

In [3]:
documents = text_splitter.split_documents(docs)


In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents[:20], OpenAIEmbeddings)


In [ ]:
query = "AN attension fucntion can be describe as mapping a query"
results = db.similarity_search(query)
results[0].page_content

In [ ]:
from langchain_community.llms import Ollama
## load Ollama LAMA2 LLm model
llm = Ollama(model="llama2")

In [ ]:
## Design a CharPrompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('''
    Answer the following question based on the provided context.
    Think step by step before providing a detailed answer.
    I will tip you $1000 if the user finds the answer helpful.
    <context>
    {context}
    </context>                                      
    Question;{input}''')


In [ ]:
## chain introduction
## Create Stuff DOcument Chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [4]:
"""
Retrivers: A retriever is an interface that returns documents given 
an unstructured query. It is more general then a vector store. 
A retriever  does not need to be able to store documents, only to return 
 (or retrieve) them. vectors stores can be used as the backbone of a retriver,
 but there are othe types of retrievers as well.

"""

retriever = db.as_retriever()
retriever


'\nRetrivers: A retriever is an interface that returns documents given \nan unstructured query. It is more general then a vector store. \nA retriever  does not need to be able to store documents, only to return \n (or retrieve) them. vectors stores can be used as the backbone of a retriver,\n but there are othe types of retrievers as well.\n\n\n'

In [ ]:

"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""

from langchain.chains import create_retrieval_chain
retrievel_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke("input":"an attention function can be describe as mapping")